# 네이버 140자평 데이터 수집
상영작 예매 순위 top15 영화에 대한 140평 데이터를 수집한다.

가져오고자 하는 요소가 1개가 아니기 때문에 각각 요소를 구분할 수 있는 값을 먼저 추출하고 
그 값을 이용해 각각의 영화를 모두 요청해서 데이터를 가져온다.

In [1]:
import bs4
import requests
import time
import pandas
import os

# 영화 평점 데이터를 담고 있을 리스트
# movie_140_comment_list = []


### 영화코드 추출

In [2]:
# 접속할 페이지의 주소
# 예매 순 첫번째 영화의 주소( 나중에 바꿔주세요 )
site = 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=176306'

In [3]:
# 요청
response = requests.get(site)
response

<Response [200]>

In [4]:
# bs4 객체를 생성
soup = bs4.BeautifulSoup(response.text, 'lxml')
soup

<!DOCTYPE html>
<html lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="no" http-equiv="imagetoolbar"/>
<title>남산의 부장들 : 네이버 영화</title>
<meta content="https://ssl.pstatic.net/imgmovie/today/naverme/naverme_profile.jpg" property="me2:image"/>
<meta content="네이버영화 남산의 부장들" property="me2:post_tag"/>
<meta content="네이버영화" property="me2:category1"/>
<meta content="남산의 부장들" property="me2:category2"/>
<meta content="남산의 부장들" property="og:title"/>
<meta content="article" property="og:type"/>
<meta content="https://movie.naver.com/movie/bi/mi/basic.nhn?code=176306" property="og:url"/>
<meta content="https://movie-phinf.pstatic.net/20200103_41/1578032244761ejKMX_JPEG/movie_image.jpg?type=m665_443_2" property="og:image"/>
<meta content="1979년 10월 26일, 중앙정보부장 김규평(이병헌)이 대한민국 대통령을 암살한다.이 사건의 40일전, 미국..." property="og:description"/>
<meta content="https://ssl.pstatic.net/imgmovie/today/naverme/naverme_profile.jpg" property="og:article:thum

In [5]:
# 상단의 영화 목록을 가지고 있는 ul 태그를 가져온다.
ul_tag= soup.find('ul', class_='top_thumb_lst')
ul_tag

<ul class="top_thumb_lst">
<li class="on" data-title="남산의 부장들"><a href="/movie/bi/mi/basic.nhn?code=176306"><img alt="남산의 부장들" height="60" onerror="src='https://ssl.pstatic.net/static/movie/2012/06/dft_img77x110.png'" src="https://movie-phinf.pstatic.net/20200103_41/1578032244761ejKMX_JPEG/movie_image.jpg?type=n77_110_2" width="42"/></a></li>
<li data-title="히트맨"><a href="/movie/bi/mi/basic.nhn?code=185838"><img alt="히트맨" height="60" onerror="src='https://ssl.pstatic.net/static/movie/2012/06/dft_img77x110.png'" src="https://movie-phinf.pstatic.net/20200122_181/15796574636309Fux2_JPEG/movie_image.jpg?type=n77_110_2" width="42"/></a></li>
<li data-title="미스터 주: 사라진 VIP"><a href="/movie/bi/mi/basic.nhn?code=177509"><img alt="미스터 주: 사라진 VIP" height="60" onerror="src='https://ssl.pstatic.net/static/movie/2012/06/dft_img77x110.png'" src="https://movie-phinf.pstatic.net/20200122_11/15796766202282GrU2_JPEG/movie_image.jpg?type=n77_110_2" width="42"/></a></li>
<li data-title="스파이 지니어스"><a href=

In [6]:
# a 태그들으르 가져온다.
a_list = ul_tag.find_all('a')
a_list

[<a href="/movie/bi/mi/basic.nhn?code=176306"><img alt="남산의 부장들" height="60" onerror="src='https://ssl.pstatic.net/static/movie/2012/06/dft_img77x110.png'" src="https://movie-phinf.pstatic.net/20200103_41/1578032244761ejKMX_JPEG/movie_image.jpg?type=n77_110_2" width="42"/></a>,
 <a href="/movie/bi/mi/basic.nhn?code=185838"><img alt="히트맨" height="60" onerror="src='https://ssl.pstatic.net/static/movie/2012/06/dft_img77x110.png'" src="https://movie-phinf.pstatic.net/20200122_181/15796574636309Fux2_JPEG/movie_image.jpg?type=n77_110_2" width="42"/></a>,
 <a href="/movie/bi/mi/basic.nhn?code=177509"><img alt="미스터 주: 사라진 VIP" height="60" onerror="src='https://ssl.pstatic.net/static/movie/2012/06/dft_img77x110.png'" src="https://movie-phinf.pstatic.net/20200122_11/15796766202282GrU2_JPEG/movie_image.jpg?type=n77_110_2" width="42"/></a>,
 <a href="/movie/bi/mi/basic.nhn?code=172780"><img alt="스파이 지니어스" height="60" onerror="src='https://ssl.pstatic.net/static/movie/2012/06/dft_img77x110.png'" sr

In [7]:
# a 태그의 href 속성 값을 가져온다.
# code_list 만들기
code_list = []

for a_tag in a_list:
    href = a_tag.attrs['href']
    str1 = href.split('=')
    code = str1[1]
    code_list.append(code)
    
code_list

['176306',
 '185838',
 '177509',
 '172780',
 '180025',
 '159866',
 '185282',
 '76960',
 '191431',
 '190336',
 '187940',
 '144694',
 '136873',
 '184571',
 '185357']

### 영화 140자 평 데이터를 추출

In [8]:
# 영화코드를 받아 응답결과를 반환하는 함수
def getHtml(code):
    site = f'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code={code}&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false'
    # 요청한다.
    response = requests.get(site)
    return response.text

In [9]:
# 전체 페이지 수를 반환하는 함수
def getPageCnt(html):
    #  bs4 객체 생성
    soup = bs4.BeautifulSoup(html, 'lxml')
    # 전체 글 개수를 가져온다.
    div_tag = soup.find('div', class_= 'score_total')
    em_tag = div_tag.find('em')
    str_total_cnt = em_tag.text.replace(',','')
    str_total_cnt = int(str_total_cnt)
    
    # 페이지 수를 계산한다.
    page_cnt = str_total_cnt // 10
    if str_total_cnt % 10 > 0:
        page_cnt = page_cnt + 1
        
    return page_cnt



In [13]:
# 현재 페이지에서 140자 평 데이터를 가져온다.
def get_140comment(code, page):
    site = f'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code={code}&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page={page}'
    # 요청한다.
    response = requests.get(site)
    # bs4 객체를 생성한다.
    soup = bs4.BeautifulSoup(response.text, 'lxml')
    # 데이터를 가지고 있는 div 태그를 가져온다.
    div_tag = soup.find('div', class_='score_result')
    # 내부에 있는 li 태그들을 가져온다.
    li_list = div_tag.find_all('li')
    
    for li_tag in li_list:
        # 별점을 가져온다.
        star_score_tag = li_tag.find('div', class_='star_score')
        em_tag = star_score_tag.find('em')
        star_score = em_tag.text
        
        # 평점글을 가져온다.
        score_reple_tag = li_tag.find('div', class_= 'score_reple')
        span_list = score_reple_tag.find_all('span')
        
    if len(span_list) > 2: # 관람객 평점
        
        score_reple = span_list[1].text.strip()
        writer_id = span_list[2].text.strip()
            
    else:                  # 일반
        score_reple = span_list[0].text.strip()
        writer_id = span_list[1].text.strip()
    # 작성 날짜
    dl_tag = score_reple_tag.find('dl')
    em_list = dl_tag.find_all('em')
    write_date = em_list[1].text
    
    reple_tuple = star_score, score_reple, writer_id, write_date
    # print(reple_tuple)
    
    # 수집한 데이터를 저장한다.
    if os.path.isfile(f'movie_data_{code}.csv') == False:
        columns = ['star_score','score_reple', 'writer_id', 'write_date']

        df1 = pandas.DataFrame([reple_tuple],columns=columns )
        df1.to_csv(f'movie_data_{code}.csv', index = False, encoding='utf-8-sig')
    else:
        df1 = pandas.DataFrame([reple_tuple])
        df1.to_csv(f'movie_data.csv_{code}', mode = 'a', index = False,header=False, encoding='utf-8-sig')        
    
    

In [14]:
for movie_code in code_list:
    html = getHtml(movie_code)
    page_cnt = getPageCnt(html)

page_cnt = 3
# 페이지 수 만큼 반복
for idx in range(1, page_cnt + 1):
    print(f'code : {movie_code}, page: {idx} / {page_cnt}')
    get_140comment(movie_code, idx)
    
print('수집 완료')

code : 185357, page: 1 / 3
code : 185357, page: 2 / 3
code : 185357, page: 3 / 3
수집 완료
